In [1]:
import tensorflow as tf
import pandas as pd
from utils_analysis import normalize_dataframe2
from math import ceil

%config IPCompleter.greedy=True



In [2]:
data = pd.read_csv("caracteristicas.csv")
print(data.dtypes)

Unnamed: 0                          int64
id                                  int64
room_type                           int64
price                               int64
minimum_nights_x                    int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
host_response_time                  int64
host_response_rate                  int64
property_type                       int64
accommodates                        int64
bathrooms                         float64
bedrooms                          float64
beds                              float64
bed_type                            int64
cleaning_fee                      float64
minimum_nights_y                    int64
maximum_nights                      int64
minimum_minimum_nights              int64
maximum_minimum_nights              int64
minimum_maximum_nights              int64
maximum_maximum_nights              int64
review_scores_rating              

In [3]:
data = data.drop(labels=['id'], axis=1)
# data = data[['price', 'number_of_reviews']]
print(data.dtypes)

Unnamed: 0                          int64
room_type                           int64
price                               int64
minimum_nights_x                    int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
host_response_time                  int64
host_response_rate                  int64
property_type                       int64
accommodates                        int64
bathrooms                         float64
bedrooms                          float64
beds                              float64
bed_type                            int64
cleaning_fee                      float64
minimum_nights_y                    int64
maximum_nights                      int64
minimum_minimum_nights              int64
maximum_minimum_nights              int64
minimum_maximum_nights              int64
maximum_maximum_nights              int64
review_scores_rating              float64
review_scores_accuracy            

In [4]:

# data, scaler = normalize_dataframe2(data)
data = data.to_numpy().astype('float32')
dataset = tf.data.Dataset.from_tensor_slices(data)


print("Shape of data: ", data.shape)

Shape of data:  (25132, 30)


In [5]:
input_dim = data.shape[1]
n_hidden_1 = 31
n_hidden_2 = 31
n_hidden_3 = 31

learning_rate = 0.01
num_steps = 100000
batch_size = 10
display_step = 1000
test_size = 1000

dataset = dataset.shuffle(buffer_size=1000)

test_dataset = dataset.take(test_size)
train_dataset = dataset.skip(test_size)

n_repeat = ceil(num_steps * batch_size /( data.shape[0] - test_size))
train_dataset = train_dataset.repeat(n_repeat)
train_dataset = train_dataset.batch(batch_size)


iter = train_dataset.make_one_shot_iterator()
X = iter.get_next()

print(n_repeat)



42


In [6]:

weigths = {
    'encoder_h1' : tf.Variable(tf.random_normal([input_dim, n_hidden_1])),
    'encoder_h2' : tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1' : tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'decoder_h2' : tf.Variable(tf.random_normal([n_hidden_3, input_dim]))    
}

biases = {
    'encoder_h1' : tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_h2' : tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_h1' : tf.Variable(tf.random_normal([n_hidden_3])),
    'decoder_h2' : tf.Variable(tf.random_normal([input_dim]))    
}

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
def red_encoder(x):
    l1 = tf.nn.relu(tf.add(tf.matmul(x, weigths['encoder_h1']), biases['encoder_h1']))
    l2 = tf.nn.sigmoid(tf.add(tf.matmul(l1, weigths['encoder_h2']), biases['encoder_h2']))
    return l2

def red_decoder(x):
    l1 = tf.nn.relu(tf.add(tf.matmul(x, weigths['decoder_h1']), biases['decoder_h1']))
    l2 = tf.nn.relu(tf.add(tf.matmul(l1, weigths['decoder_h2']), biases['decoder_h2']))
    return l2

In [8]:
encoder = red_encoder(X)
decoder = red_decoder(encoder)

y_true = X
y_pred = decoder

loss = tf.reduce_sum(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate)
optimizer = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [9]:


sess = tf.Session()

sess.run(init)


for i in range(1, num_steps + 1):
    _, l = sess.run([optimizer, loss])
    if i % display_step == 0 or i == 1:
        print('Step %i: Minibatch Loss: %f' % (i, l/batch_size))

Step 1: Minibatch Loss: 4495960.000000
Step 1000: Minibatch Loss: 3966454.400000
Step 2000: Minibatch Loss: 4301632.400000
Step 3000: Minibatch Loss: 12340022.400000
Step 4000: Minibatch Loss: 6400575.200000
Step 5000: Minibatch Loss: 983184793.600000
Step 6000: Minibatch Loss: 5565180.400000
Step 7000: Minibatch Loss: 2897680.600000
Step 8000: Minibatch Loss: 10424424.000000
Step 9000: Minibatch Loss: 4239021.200000
Step 10000: Minibatch Loss: 169056000.000000
Step 11000: Minibatch Loss: 14423502.400000
Step 12000: Minibatch Loss: 5252931.200000
Step 13000: Minibatch Loss: 5476952.000000
Step 14000: Minibatch Loss: 8826670.400000
Step 15000: Minibatch Loss: 20395947.200000
Step 16000: Minibatch Loss: 2898830.000000
Step 17000: Minibatch Loss: 1785059328.000000
Step 18000: Minibatch Loss: 6310416.400000
Step 19000: Minibatch Loss: 2312229.600000
Step 20000: Minibatch Loss: 8955646.400000
Step 21000: Minibatch Loss: 4738771.200000
Step 22000: Minibatch Loss: 350724684.800000
Step 23000:

In [10]:
test_dataset = test_dataset.batch(1)
iter_test = test_dataset.make_one_shot_iterator()
I = iter_test.get_next()

# I = tf.placeholder(tf.float32, shape=([None, input_dim]))

d_encoder = red_encoder(I)
d_decoder = red_decoder(d_encoder)
perdida = tf.reduce_sum(tf.pow(d_decoder - I, 2))

In [11]:

v, l, i = sess.run([d_decoder, perdida, I])
print(v.shape)
print(l)


# i_r = scaler.inverse_transform(v)
# o_r = scaler.inverse_transform(i)
# print(i_r)
# print(o_r)

print(v)
print(i)

(1, 30)
789712800.0
[[29579.445        0.         135.64523      0.           0.
      0.           0.           0.          87.56758      0.
      0.           0.           0.           0.           0.
     48.167988     0.         632.5087       0.           0.
    627.5918     627.57294      0.           0.           0.
      0.           0.           0.           0.           0.      ]]
[[1.485e+03 0.000e+00 1.700e+02 3.000e+01 2.900e+02 3.940e+00 1.000e+00
  3.000e+00 1.000e+02 6.000e+00 2.000e+00 1.000e+00 1.000e+00 1.000e+00
  4.000e+00 5.000e+01 3.000e+01 3.050e+02 3.000e+01 3.000e+01 3.050e+02
  3.050e+02 9.900e+01 1.000e+01 1.000e+01 1.000e+01 1.000e+01 1.000e+01
  1.000e+01 2.000e+00]]


In [12]:
print(v - i)
# print(i_r - o_r)

[[ 2.8094445e+04  0.0000000e+00 -3.4354767e+01 -3.0000000e+01
  -2.9000000e+02 -3.9400001e+00 -1.0000000e+00 -3.0000000e+00
  -1.2432419e+01 -6.0000000e+00 -2.0000000e+00 -1.0000000e+00
  -1.0000000e+00 -1.0000000e+00 -4.0000000e+00 -1.8320122e+00
  -3.0000000e+01  3.2750873e+02 -3.0000000e+01 -3.0000000e+01
   3.2259180e+02  3.2257294e+02 -9.9000000e+01 -1.0000000e+01
  -1.0000000e+01 -1.0000000e+01 -1.0000000e+01 -1.0000000e+01
  -1.0000000e+01 -2.0000000e+00]]
